In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("database.sqlite")

In [3]:
master = pd.read_sql("select * from Emails e left join EmailReceivers r on e.Id = r.EmailId where r.PersonId=80", con)

In [4]:
master.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText,Id,EmailId,PersonId
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,,UNCLASSIFIED\nU.S. Department of State\nCase N...,1,1,80
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,,,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...,2,2,80
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,"Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...,4,3,80
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,"Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,,UNCLASSIFIED\nU.S. Department of State\nCase N...,5,4,80
4,7,C05739560,"ANTI-MUSLIM FILM DIRECTOR IN HIDING, FOLLOWING...",H,"Mills, Cheryl D",32,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739560...,F-2015-04841,...,"Wednesday, September 12, 2012 4:00 PM",F-2015-04841,C05739560,05/13/2015,RELEASE IN FULL,,UNCLASSIFIED\nU.S. Department of State\nCase N...,8,7,80


In [5]:
df_annotated = pd.read_csv('project-16038-at-2022-10-02-08-09-4d73ddb7.csv')

In [6]:
len(df_annotated)

2044

In [7]:
df_annotated.head()

,dbId,text,docNumber,id,sentiment,annotator,annotation_id,created_at,updated_at,lead_time,agreement
0,1,<b>Subject:</b> <i>WOW</i><br/><b>Body</b>: no...,C05739545,26842910,Ignore,amir.nagri@gmail.com,10618178,2022-10-02T07:25:44.810960Z,2022-10-02T07:25:44.810976Z,5.902,100.0
1,2,<b>Subject:</b> <i>H: LATEST: HOW SYRIA IS AID...,C05739546,26842911,Schedule,amir.nagri@gmail.com,10618085,2022-10-02T07:12:03.780777Z,2022-10-02T07:12:03.780791Z,16.611,100.0
2,3,<b>Subject:</b> <i>CHRIS STEVENS</i><br/><b>Bo...,C05739547,26842912,Ignore,amir.nagri@gmail.com,10618179,2022-10-02T07:25:48.808462Z,2022-10-02T07:25:48.808476Z,2.897,100.0
3,4,<b>Subject:</b> <i>CAIRO CONDEMNATION - FINAL<...,C05739550,26842913,Ignore,amir.nagri@gmail.com,10618180,2022-10-02T07:25:54.488750Z,2022-10-02T07:25:54.488764Z,4.662,100.0
4,7,<b>Subject:</b> <i>ANTI-MUSLIM FILM DIRECTOR I...,C05739560,26842914,Do,amir.nagri@gmail.com,10618181,2022-10-02T07:25:59.721920Z,2022-10-02T07:25:59.721942Z,4.201,100.0


In [8]:
df_annotated.drop_duplicates(subset=['docNumber'], inplace=True, keep='last')

In [9]:
# drop rows with sentiment ignore or NA
drop_cond = (df_annotated['sentiment'] == 'Ignore') | df_annotated['sentiment'].isna()
df_annotated.drop(df_annotated[drop_cond].index, inplace=True)

In [10]:
len(df_annotated)

1501

In [11]:
df_with_annot = master.join(df_annotated.set_index('docNumber'), on='DocNumber')

In [12]:
# drop rows that are not annotated
df_annot_rows = df_with_annot[df_with_annot['sentiment'].notna()].copy()

In [13]:
df_annot_rows.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,dbId,text,id,sentiment,annotator,annotation_id,created_at,updated_at,lead_time,agreement
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,,,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,2.0,<b>Subject:</b> <i>H: LATEST: HOW SYRIA IS AID...,26842911.0,Schedule,amir.nagri@gmail.com,10618085.0,2022-10-02T07:12:03.780777Z,2022-10-02T07:12:03.780791Z,16.611,100.0
4,7,C05739560,"ANTI-MUSLIM FILM DIRECTOR IN HIDING, FOLLOWING...",H,"Mills, Cheryl D",32,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739560...,F-2015-04841,...,7.0,<b>Subject:</b> <i>ANTI-MUSLIM FILM DIRECTOR I...,26842914.0,Do,amir.nagri@gmail.com,10618181.0,2022-10-02T07:25:59.721920Z,2022-10-02T07:25:59.721942Z,4.201,100.0
6,10,C05739563,MORE ON LIBYA,H,,,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739563...,F-2015-04841,...,10.0,<b>Subject:</b> <i>MORE ON LIBYA</i><br/><b>Bo...,26842916.0,Schedule,amir.nagri@gmail.com,10584767.0,2022-09-30T14:34:56.928113Z,2022-09-30T14:34:56.928128Z,8.167,100.0
7,11,C05739564,ABZ AN HBJ ON LIBYA AND WEST BANK/GAZA,H,"Sullivan, Jacob J",87,2011-03-13T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739564...,F-2015-04841,...,11.0,<b>Subject:</b> <i>ABZ AN HBJ ON LIBYA AND WES...,26842917.0,Do,amir.nagri@gmail.com,10618183.0,2022-10-02T07:26:09.695266Z,2022-10-02T07:26:09.695279Z,5.205,100.0
9,13,C05739567,HEY,H,"Sullivan, Jacob J",87,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739567...,F-2015-04841,...,13.0,<b>Subject:</b> <i>HEY</i><br/><b>Body</b>:<pr...,26842919.0,Delete,amir.nagri@gmail.com,10577804.0,2022-09-30T07:59:52.201247Z,2022-09-30T07:59:52.201262Z,3.605,100.0


In [14]:
len(df_annot_rows)

1502

In [15]:
df_annot_rows['clean_subject'] = df_annot_rows['MetadataSubject'].str.strip().str.replace("\n", " ").str.replace("\t","  ").str.title()

In [16]:
df_annot_rows['clean_body'] = df_annot_rows['ExtractedBodyText'].str.strip().str.replace("\n", " ").str.replace("\t","  ")

In [17]:
# prepare fine tuning string
df_annot_rows['train_txt'] = df_annot_rows['clean_subject'] + " <body> " + df_annot_rows['clean_body']

In [18]:
# select columns to export
df_export = df_annot_rows[['train_txt', 'sentiment']].copy()

In [19]:
df_export.head()

,train_txt,sentiment
1,H: Latest: How Syria Is Aiding Qaddafi And Mor...,Schedule
4,"Anti-Muslim Film Director In Hiding, Following...",Do
6,"More On Libya <body> B6 Wednesday, September 1...",Schedule
7,Abz An Hbj On Libya And West Bank/Gaza <body> ...,Do
9,Hey <body> Fyi,Delete


In [20]:
df_export['sentiment'].unique()

array(['Schedule', 'Do', 'Delete', 'Delegate'], dtype=object)

In [21]:
# update the sentiments to match email labels
df_export.loc[df_export['sentiment'] == 'Do', 'sentiment'] = 'Now'
df_export.loc[df_export['sentiment'] == 'Schedule', 'sentiment'] = 'Later'
df_export.loc[df_export['sentiment'] == 'Delete', 'sentiment'] = 'Discard'
df_export['sentiment'].unique()

array(['Later', 'Now', 'Discard', 'Delegate'], dtype=object)

In [22]:
df_export.drop_duplicates(keep='last', inplace=True)

In [23]:
# export csv to fine tune the model
df_export.to_csv('train_classify_finetune.csv', index=False, header=False)

In [24]:
!head -20 train_classify_finetune.csv

"H: Latest: How Syria Is Aiding Qaddafi And More... Sid <body> B6 Thursday, March 3, 2011 9:45 PM H: Latest How Syria is aiding Qaddafi and more... Sid hrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx March 3, 2011 For: Hillary",Later
"Anti-Muslim Film Director In Hiding, Following Libya, Egypt Violence <body> ",Now
"More On Libya <body> B6 Wednesday, September 12, 2012 6:16 PM Fwd: more on libya Libya 37 sept 12 12.docx Sending direct. Just in. Sent from my Verizon Wireless 4G LTE DRUID",Later
Abz An Hbj On Libya And West Bank/Gaza <body> Fyi B6 — —,Now
Hey <body> Fyi,Discard
"Thank You <body> I asked to attend your svtc today with Embassy Tripoli, because had first met so many of that staff when I went with you from Malta to Tripoli for the reopening of our Embassy. Today's deaths hit me much harder than I would have guessed. I am always proud to serve under you, but never have your words been more meaningful than on today's svtc. Every day of your ten

---